In [9]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, plot_roc_curve, make_scorer, f1_score, roc_auc_score
from sklearn import preprocessing
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, LeaveOneGroupOut, PredefinedSplit, GridSearchCV
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [ ]:
for i in range (1, 9):
    

In [36]:
def merge_dataframes_for_train(user_path, file_type, count, not_use_postfixes = []):
    i = 0
    df_list = []
    used_postfixes = []
    for file in os.listdir(user_path):
        if i == count:
            break
        if file.find(file_type) != -1 and file[-11:] not in not_use_postfixes:
            df_list.append(pd.read_csv(os.path.join(user_path, file)))
            used_postfixes.append(file[-11:])
            i += 1
            
    return pd.concat(df_list, ignore_index=True), used_postfixes       

In [37]:
train_df, used = merge_dataframes_for_train("..\\..\\scripts\\_feature_events\\15min\\rolling\\60s\\user_1", "bt", 4)

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [38]:
train_df = train_df.rename(columns={'user_user_agg':'user'})
train_df = train_df.drop(['timestamp'], axis=1)

In [39]:
def drop_bad_rows(df):
    bad_rows = set()
    for col in df.columns:
        if col != "user":
            for user in df.user.unique():
                print(col, user)
                for x in list(
                    df.loc[df.user == user, :][
                        np.abs(stats.zscore(df.loc[df.user == user, col])) > 3
                    ].index
                ):
                    bad_rows.add(x)

            for x in list(df[col][np.abs(stats.zscore(df[col])) > 3].index):
                bad_rows.add(x)

    df.drop(list(bad_rows), axis=0, inplace=True)

    return df

In [66]:
train_df

,count_kurt,count_mean,count_median,count_skew,count_speed_mean_mad,count_speed_mean_mean,count_speed_mean_skew,count_speed_median_mad,count_speed_median_mean,count_speed_median_skew,...,occured_devices_count_median_kurt,occured_devices_count_median_mad,occured_devices_count_median_mean,occured_devices_count_median_median,occured_devices_count_median_skew,occured_devices_count_median_var,occured_devices_count_var_mad,occured_devices_count_var_mean,occured_devices_count_var_skew,user
3,-3.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,-3.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0
4,-3.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,-3.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0
5,-3.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,-3.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0
6,-3.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,-3.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0
7,-3.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,-3.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0
8,-3.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,-3.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0
9,-3.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,-3.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0
10,-3.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,-3.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0
11,-3.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,-3.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0
12,-3.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,-3.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0


In [67]:
test_df

,count_mean,count_var,count_median,count_skew,count_kurt,le_connectable_mean,le_connectable_var,le_connectable_median,le_connectable_skew,le_connectable_kurt,...,count_speed_mean_mad,count_speed_mean_skew,count_speed_median_mean,count_speed_median_mad,count_speed_median_skew,count_speed_var_mean,count_speed_var_mad,count_speed_var_skew,user,events_count_sum
0,0.000000,0.000000,0.0,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,...,0.750000,-1.709126,-0.500000,0.833333,-1.788854,0.500000,0.833333,1.788854,1.0,0.0
1,0.000000,0.000000,0.0,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,...,0.416667,-2.223974,-0.250000,0.458333,-3.015113,0.500000,0.833333,1.788854,1.0,0.0
2,0.000000,0.000000,0.0,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,...,0.152778,-3.015113,0.000000,0.000000,0.000000,0.250000,0.458333,3.015113,1.0,0.0
3,0.000000,0.000000,0.0,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0
4,0.000000,0.000000,0.0,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0
5,0.000000,0.000000,0.0,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0
6,0.000000,0.000000,0.0,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0
7,0.000000,0.000000,0.0,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0
8,0.000000,0.000000,0.0,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0
9,0.000000,0.000000,0.0,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0


In [40]:
train_df = drop_bad_rows(train_df)

count_kurt 1.0
count_kurt 0.0
count_mean 1.0
count_mean 0.0
count_median 1.0
count_median 0.0
count_skew 1.0
count_skew 0.0
count_speed_mean_mad 1.0
count_speed_mean_mad 0.0
count_speed_mean_mean 1.0
count_speed_mean_mean 0.0
count_speed_mean_skew 1.0
count_speed_mean_skew 0.0
count_speed_median_mad 1.0
count_speed_median_mad 0.0
count_speed_median_mean 1.0
count_speed_median_mean 0.0
count_speed_median_skew 1.0
count_speed_median_skew 0.0
count_speed_var_mad 1.0
count_speed_var_mad 0.0
count_speed_var_mean 1.0
count_speed_var_mean 0.0
count_speed_var_skew 1.0
count_speed_var_skew 0.0
count_var 1.0
count_var 0.0
disappeared_devices_count_mean_mad 1.0
disappeared_devices_count_mean_mad 0.0
disappeared_devices_count_mean_mean 1.0
disappeared_devices_count_mean_mean 0.0
disappeared_devices_count_mean_skew 1.0
disappeared_devices_count_mean_skew 0.0
disappeared_devices_count_median_mad 1.0
disappeared_devices_count_median_mad 0.0
disappeared_devices_count_median_mean 1.0
disappeared_device

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  


le_level_skew 1.0
le_level_skew 0.0
le_level_var 1.0
le_level_var 0.0
occur_speed_mean_mad 1.0
occur_speed_mean_mad 0.0
occur_speed_mean_mean 1.0
occur_speed_mean_mean 0.0
occur_speed_mean_skew 1.0
occur_speed_mean_skew 0.0
occur_speed_median_mad 1.0
occur_speed_median_mad 0.0
occur_speed_median_mean 1.0
occur_speed_median_mean 0.0
occur_speed_median_skew 1.0
occur_speed_median_skew 0.0
occur_speed_var_mad 1.0
occur_speed_var_mad 0.0
occur_speed_var_mean 1.0
occur_speed_var_mean 0.0
occur_speed_var_skew 1.0
occur_speed_var_skew 0.0
occured_devices_count_mean_kurt 1.0
occured_devices_count_mean_kurt 0.0
occured_devices_count_mean_mad 1.0
occured_devices_count_mean_mad 0.0
occured_devices_count_mean_mean 1.0
occured_devices_count_mean_mean 0.0
occured_devices_count_mean_median 1.0
occured_devices_count_mean_median 0.0
occured_devices_count_mean_skew 1.0
occured_devices_count_mean_skew 0.0
occured_devices_count_mean_var 1.0
occured_devices_count_mean_var 0.0
occured_devices_count_median_k

In [58]:
test_df, _ = merge_dataframes_for_train("..\\..\\scripts\\_feature_events\\15min\\rolling\\60s\\user_1", "bt", 3, used)
test_df = test_df.rename(columns={'user_user_agg':'user'})
test_df = test_df.drop(['timestamp'], axis=1)

In [59]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [54]:
dataset = train_df.to_numpy().copy()
X = dataset[:, :-1]
y = dataset[:, -1]

In [55]:
C = 1
kernel = 'rbf'
degree = 1
gamma = 5

In [56]:
model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)

In [57]:
model.fit(X, y)

SVC(C=1, degree=1, gamma=5)

In [60]:
dataset = test_df.to_numpy().copy()
X_test = dataset[:, :-1]
y_test = dataset[:, -1]

In [61]:
y_t = y_test.copy()
predict = model.predict(X_test).copy()
predict_pr = model.predict_proba(X_test).copy()

ValueError: X.shape[1] = 61 should be equal to 67, the number of features at training time